# The Domestic Movie Box Office Before, After, and During the COVID-19 Pandemic
#### *Author*: Austin Villegas
#### *Last Updated*: 05/23/2023

# Table of Contents

1. [Problem Statement](#problem_statement)
    * [Definitions](#definitions)
    * [Goals](#goals)
2. [Analysis](#analysis)   
    * [Imports](#imports)
    * [Data Cleaning](#data_cleaning)
    * [Test for Normality](#test_for_normality)
    * [z-Test vs. t-Test](#z_test_vs_t_test)
    * [Confidence Intervals](#confidence_intervals)
    * [Analysis of Variance (ANOVA)](#anova)
3. [Conclusion](#conclusion)

<a id="problem_statement"></a> 
# Problem Statement 

<a id="definitions"></a> 
### Definitions

<a id="goals"></a> 
### Goals

<a id="analysis"></a> 
# Analysis 

<a id="imports"></a> 
### Imports

<a id="data_cleaning"></a> 
### Data Cleaning

<a id="test_for_normality"></a> 
### Test for Normality

<a id="z_test_vs_t_test"></a> 
### z-Test vs t-Test

<a id="confidence_intervals"></a> 
### Confidence Intervals

<a id="anova"></a> 
### Analysis of Variance (ANOVA)

<a id="conclusion"></a> 
# Conclusion 